In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [3]:
df=pd.read_csv('data/data.csv')
df.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
0,0xcdcd,DEHRES17DEL01,36.0,4.2,30.327968,78.046106,30.397968,78.116106,12-02-2022,21:55,22:10,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46
1,0xd987,KOCRES16DEL01,21.0,4.7,10.003064,76.307589,10.043064,76.347589,13-02-2022,14:55,15:05,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23
2,0x2784,PUNERES13DEL03,23.0,4.7,18.562450,73.916619,18.652450,74.006619,04-03-2022,17:30,17:40,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21
3,0xc8b6,LUDHRES15DEL02,34.0,4.3,30.899584,75.809346,30.919584,75.829346,13-02-2022,09:20,09:30,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20
4,0xdb64,KNPRES14DEL02,24.0,4.7,26.463504,80.372929,26.593504,80.502929,14-02-2022,19:50,20:05,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41


In [12]:
df['City'].value_counts()

Metropolitian    34087
Urban            10133
Semi-Urban         164
Name: City, dtype: int64

In [4]:
df.shape

(45584, 20)

In [5]:
df.drop(columns=['Order_Date','ID'],axis=1,inplace=True)

In [6]:
from pandas_profiling import ProfileReport
pf = ProfileReport(df)

C:\Users\HP\AppData\Local\Temp\ipykernel_13748\3507663625.py:1: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


In [7]:
# pf.to_widgets()

In [8]:
#As we can notice high correlation between (Delivery_location_latitude,Restaurant_latitude) and (Restaurant_longitude,Delivery_location_longitude) so we can remove each feature from both of the groups
df.drop(columns=['Restaurant_latitude','Restaurant_longitude'],inplace=True)

In [9]:
df.isnull().sum()

Delivery_person_ID                0
Delivery_person_Age            1854
Delivery_person_Ratings        1908
Delivery_location_latitude        0
Delivery_location_longitude       0
Time_Orderd                    1731
Time_Order_picked                 0
Weather_conditions              616
Road_traffic_density            601
Vehicle_condition                 0
Type_of_order                     0
Type_of_vehicle                   0
multiple_deliveries             993
Festival                        228
City                           1200
Time_taken (min)                  0
dtype: int64

In [10]:
#Segregate numerical columns and categorical columns
numerical_columns = df.columns[df.dtypes!='object']
categorical_columns = df.columns[df.dtypes=='object']
print('numerical_columns',numerical_columns)
print('categorical_columns',categorical_columns)

numerical_columns Index(['Delivery_person_Age', 'Delivery_person_Ratings',
       'Delivery_location_latitude', 'Delivery_location_longitude',
       'Vehicle_condition', 'multiple_deliveries', 'Time_taken (min)'],
      dtype='object')
categorical_columns Index(['Delivery_person_ID', 'Time_Orderd', 'Time_Order_picked',
       'Weather_conditions', 'Road_traffic_density', 'Type_of_order',
       'Type_of_vehicle', 'Festival', 'City'],
      dtype='object')


In [11]:
from sklearn.impute import SimpleImputer
num_imputer = SimpleImputer(strategy='median')
num_imputer.fit(df[numerical_columns])
df[numerical_columns] = num_imputer.transform(df[numerical_columns])

In [12]:
df[numerical_columns].isnull().sum()

Delivery_person_Age            0
Delivery_person_Ratings        0
Delivery_location_latitude     0
Delivery_location_longitude    0
Vehicle_condition              0
multiple_deliveries            0
Time_taken (min)               0
dtype: int64

In [13]:
df[categorical_columns].isnull().sum()

Delivery_person_ID         0
Time_Orderd             1731
Time_Order_picked          0
Weather_conditions       616
Road_traffic_density     601
Type_of_order              0
Type_of_vehicle            0
Festival                 228
City                    1200
dtype: int64

In [14]:
cat_imputer = SimpleImputer(strategy='most_frequent')
df[categorical_columns] = cat_imputer.fit_transform(df[categorical_columns])

In [15]:
df[categorical_columns].isnull().sum()

Delivery_person_ID      0
Time_Orderd             0
Time_Order_picked       0
Weather_conditions      0
Road_traffic_density    0
Type_of_order           0
Type_of_vehicle         0
Festival                0
City                    0
dtype: int64

In [16]:
df['Delivery_person_Age'] = df['Delivery_person_Age'].astype(int)
df['multiple_deliveries'] = df['multiple_deliveries'].astype(int)

In [17]:
df['Time_Orderd_min'] = df['Time_Orderd'].str.split(':').apply(lambda x: int(x[1] if len(x)>1 else int(float(x[0])*60)))
df['Time_Order_picked_min'] = df['Time_Order_picked'].str.split(':').apply(lambda x: int(x[1] if len(x)>1 else int(float(x[0])*60)))

In [18]:
# Dropping Time_Orderd,Time_Order_picked
df.drop(columns=['Time_Orderd','Time_Order_picked'],axis=1,inplace=True)

In [19]:
df.isnull().sum()

Delivery_person_ID             0
Delivery_person_Age            0
Delivery_person_Ratings        0
Delivery_location_latitude     0
Delivery_location_longitude    0
Weather_conditions             0
Road_traffic_density           0
Vehicle_condition              0
Type_of_order                  0
Type_of_vehicle                0
multiple_deliveries            0
Festival                       0
City                           0
Time_taken (min)               0
Time_Orderd_min                0
Time_Order_picked_min          0
dtype: int64

In [20]:
df.dtypes

Delivery_person_ID              object
Delivery_person_Age              int32
Delivery_person_Ratings        float64
Delivery_location_latitude     float64
Delivery_location_longitude    float64
Weather_conditions              object
Road_traffic_density            object
Vehicle_condition              float64
Type_of_order                   object
Type_of_vehicle                 object
multiple_deliveries              int32
Festival                        object
City                            object
Time_taken (min)               float64
Time_Orderd_min                  int64
Time_Order_picked_min            int64
dtype: object

In [21]:
#Segregate numerical columns and categorical columns
numerical_columns = df.columns[df.dtypes!='object']
categorical_columns = df.columns[df.dtypes=='object']
print('numerical_columns',numerical_columns)
print('categorical_columns',categorical_columns)

numerical_columns Index(['Delivery_person_Age', 'Delivery_person_Ratings',
       'Delivery_location_latitude', 'Delivery_location_longitude',
       'Vehicle_condition', 'multiple_deliveries', 'Time_taken (min)',
       'Time_Orderd_min', 'Time_Order_picked_min'],
      dtype='object')
categorical_columns Index(['Delivery_person_ID', 'Weather_conditions', 'Road_traffic_density',
       'Type_of_order', 'Type_of_vehicle', 'Festival', 'City'],
      dtype='object')


In [22]:
cat_df=df[categorical_columns]
cat_df.head()

,Delivery_person_ID,Weather_conditions,Road_traffic_density,Type_of_order,Type_of_vehicle,Festival,City
0,DEHRES17DEL01,Fog,Jam,Snack,motorcycle,No,Metropolitian
1,KOCRES16DEL01,Stormy,High,Meal,motorcycle,No,Metropolitian
2,PUNERES13DEL03,Sandstorms,Medium,Drinks,scooter,No,Metropolitian
3,LUDHRES15DEL02,Sandstorms,Low,Buffet,motorcycle,No,Metropolitian
4,KNPRES14DEL02,Fog,Jam,Snack,scooter,No,Metropolitian


In [23]:
print('Weather_conditions',cat_df['Weather_conditions'].unique())
print('Road_traffic_density',cat_df['Road_traffic_density'].unique())
print('Type_of_order',cat_df['Type_of_order'].unique())
print('Type_of_vehicle',cat_df['Type_of_vehicle'].unique())
print('Festival',cat_df['Festival'].unique())
print('City',cat_df['City'].unique())

Weather_conditions ['Fog' 'Stormy' 'Sandstorms' 'Windy' 'Cloudy' 'Sunny']
Road_traffic_density ['Jam' 'High' 'Medium' 'Low']
Type_of_order ['Snack' 'Meal' 'Drinks' 'Buffet']
Type_of_vehicle ['motorcycle' 'scooter' 'electric_scooter' 'bicycle']
Festival ['No' 'Yes']
City ['Metropolitian' 'Urban' 'Semi-Urban']


In [24]:
categories = ['Fog','Stormy','Sandstorms','Windy','Cloudy','Sunny'],['Jam', 'High', 'Medium', 'Low'],['Drinks','Snack','Meal','Buffet'],['bicycle','electric_scooter','scooter','motorcycle'],['Yes','No'],['Semi-Urban','Urban','Metropolitian']
ordinal_cols = df[categorical_columns].drop(columns='Delivery_person_ID',axis=1)

In [25]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder(categories=list(categories))
encoded_columns = encoder.fit_transform(ordinal_cols)
encoded_df = pd.DataFrame(encoded_columns,columns=ordinal_cols.columns).astype(int)

In [26]:
encoded_df.head()

,Weather_conditions,Road_traffic_density,Type_of_order,Type_of_vehicle,Festival,City
0,0,0,1,3,1,2
1,1,1,2,3,1,2
2,2,2,0,2,1,2
3,2,3,3,3,1,2
4,0,0,1,2,1,2


In [27]:
#Merge encoded_df and numerical_df
final_df = pd.concat([df[numerical_columns],encoded_df],axis=1)
final_df.head()

,Delivery_person_Age,Delivery_person_Ratings,Delivery_location_latitude,Delivery_location_longitude,Vehicle_condition,multiple_deliveries,Time_taken (min),Time_Orderd_min,Time_Order_picked_min,Weather_conditions,Road_traffic_density,Type_of_order,Type_of_vehicle,Festival,City
0,36,4.2,30.397968,78.116106,2.0,3,46.0,55,10,0,0,1,3,1,2
1,21,4.7,10.043064,76.347589,1.0,1,23.0,55,5,1,1,2,3,1,2
2,23,4.7,18.652450,74.006619,1.0,1,21.0,30,40,2,2,0,2,1,2
3,34,4.3,30.919584,75.829346,0.0,0,20.0,20,30,2,3,3,3,1,2
4,24,4.7,26.593504,80.502929,1.0,1,41.0,50,5,0,0,1,2,1,2


First attempt

In [28]:
X = final_df.drop(columns=['Time_taken (min)'])
y = final_df['Time_taken (min)']

In [29]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

In [30]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
linear = LinearRegression()
linear.fit(X_train,y_train)

LinearRegression()

In [31]:
y_pred = linear.predict(X_test)
print('mean_squared_error',mean_squared_error(y_pred,y_test))
print('mean_absolute_error',mean_absolute_error(y_pred,y_test))
print('r2_score',r2_score(y_pred,y_test))

mean_squared_error 44.284784381414774
mean_absolute_error 5.25541970388441
r2_score -0.03186571977881436


In [32]:
final_df.head()

,Delivery_person_Age,Delivery_person_Ratings,Delivery_location_latitude,Delivery_location_longitude,Vehicle_condition,multiple_deliveries,Time_taken (min),Time_Orderd_min,Time_Order_picked_min,Weather_conditions,Road_traffic_density,Type_of_order,Type_of_vehicle,Festival,City
0,36,4.2,30.397968,78.116106,2.0,3,46.0,55,10,0,0,1,3,1,2
1,21,4.7,10.043064,76.347589,1.0,1,23.0,55,5,1,1,2,3,1,2
2,23,4.7,18.652450,74.006619,1.0,1,21.0,30,40,2,2,0,2,1,2
3,34,4.3,30.919584,75.829346,0.0,0,20.0,20,30,2,3,3,3,1,2
4,24,4.7,26.593504,80.502929,1.0,1,41.0,50,5,0,0,1,2,1,2


In [33]:
from pandas_profiling import ProfileReport
pf = ProfileReport(final_df)

In [34]:
# pf.to_widgets()

In [35]:
#Calculating time difference order and pickup
x = []
for i in range(len(final_df)):
    if final_df['Time_Order_picked_min'][i] < final_df['Time_Orderd_min'][i]:
        x.append(final_df['Time_Order_picked_min'][i] + (60-final_df['Time_Orderd_min'][i]))
    else:
        x.append(final_df['Time_Order_picked_min'][i] - final_df['Time_Orderd_min'][i])
final_df['Time_diff_bw_order_and_pickup'] = x

Second attempt

In [36]:
X = final_df.drop(columns=['Time_taken (min)'])
y = final_df['Time_taken (min)']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

In [37]:
linear2 = LinearRegression()
linear2.fit(X_train,y_train)

LinearRegression()

In [38]:
y_pred = linear2.predict(X_test)
print('mean_squared_error',mean_squared_error(y_pred,y_test))
print('mean_absolute_error',mean_absolute_error(y_pred,y_test))
print('r2_score',r2_score(y_pred,y_test))

mean_squared_error 44.46842546967338
mean_absolute_error 5.254784677977792
r2_score -0.023226964574704967


Third Attempt

In [39]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
final_df[:] = scaler.fit_transform(final_df)

C:\Users\HP\AppData\Local\Temp\ipykernel_13748\1028175497.py:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  final_df[:] = scaler.fit_transform(final_df)


In [40]:
X = final_df.drop(columns=['Time_taken (min)'])
y = final_df['Time_taken (min)']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

In [41]:
linear3 = LinearRegression()
linear3.fit(X_train,y_train)

LinearRegression()

In [42]:
y_pred = linear3.predict(X_test)
print('mean_squared_error',mean_squared_error(y_pred,y_test))
print('mean_absolute_error',mean_absolute_error(y_pred,y_test))
print('r2_score',r2_score(y_pred,y_test))

mean_squared_error 0.022380448065083415
mean_absolute_error 0.11845937091922651
r2_score 0.026157927850548157


Logistic regression First attempt Tommorow's task

In [43]:
from sklearn.svm import SVR
logistic = SVR()
logistic.fit(X_train,y_train)

SVR()

In [44]:
y_pred = logistic.predict(X_test)
print('mean_squared_error',mean_squared_error(y_pred,y_test))
print('mean_absolute_error',mean_absolute_error(y_pred,y_test))
print('r2_score',r2_score(y_pred,y_test))

mean_squared_error 0.01690961615008893
mean_absolute_error 0.10244129247786882
r2_score 0.4032273690453333


In [45]:
final_df

,Delivery_person_Age,Delivery_person_Ratings,Delivery_location_latitude,Delivery_location_longitude,Vehicle_condition,multiple_deliveries,Time_taken (min),Time_Orderd_min,Time_Order_picked_min,Weather_conditions,Road_traffic_density,Type_of_order,Type_of_vehicle,Festival,City,Time_diff_bw_order_and_pickup
0,0.600000,0.64,0.978866,0.882022,0.666667,1.000000,0.818182,0.9,0.090909,0.0,0.000000,0.333333,1.000000,1.0,1.0,0.254237
1,0.171429,0.74,0.323188,0.862051,0.333333,0.333333,0.295455,0.9,0.000000,0.2,0.333333,0.666667,1.000000,1.0,1.0,0.169492
2,0.228571,0.74,0.600516,0.835615,0.333333,0.333333,0.250000,0.4,0.636364,0.4,0.666667,0.000000,0.666667,1.0,1.0,0.169492
3,0.542857,0.66,0.995668,0.856199,0.000000,0.000000,0.227273,0.2,0.454545,0.4,1.000000,1.000000,1.000000,1.0,1.0,0.169492
4,0.257143,0.74,0.856315,0.908976,0.333333,0.333333,0.704545,0.8,0.000000,0.0,0.000000,0.333333,0.666667,1.0,1.0,0.254237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45579,0.428571,0.76,0.866585,0.855915,0.333333,0.000000,0.500000,0.5,0.727273,0.6,0.333333,0.666667,1.000000,1.0,1.0,0.169492
45580,0.171429,0.72,0.001933,0.000678,0.000000,0.333333,0.590909,0.9,0.090909,0.6,0.000000,1.000000,1.000000,1.0,1.0,0.254237
45581,0.428571,0.78,0.420125,0.906373,0.333333,0.000000,0.136364,0.8,0.000000,0.8,1.000000,0.000000,0.666667,1.0,1.0,0.254237
45582,0.142857,0.74,0.355358,0.869715,0.000000,0.333333,0.363636,0.5,0.636364,0.8,0.333333,0.333333,1.000000,1.0,1.0,0.084746
